In [1]:
import pandas as pd
import graphlab

In [2]:
#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/Users/rushabh/Rushabh/Untitled Folder/ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('/Users/rushabh/Rushabh/Untitled Folder/ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')

#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('/Users/rushabh/Rushabh/Untitled Folder/ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

In [3]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_train = pd.read_csv('/Users/rushabh/Rushabh/Untitled Folder/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('/Users/rushabh/Rushabh/Untitled Folder/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [4]:
#Converting the train and test data to be used by graphlab

train_data = graphlab.SFrame(ratings_train)
test_data = graphlab.SFrame(ratings_test)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1513048803.log


This non-commercial license of GraphLab Create for academic use is assigned to rss66@njit.edu and will expire on December 05, 2018.


In [5]:
#Creating a popularity model

popularity_model = graphlab.popularity_recommender.create(train_data, user_id='user_id', item_id='movie_id', target='rating')

Recsys training: model = popularity

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

Data prepared in: 0.12107s

90570 observations to process; with 1680 unique items.

In [11]:
#Get recommendations for 5 users and print them

popularity_recomm = popularity_model.recommend(users=range(20,26),k=5)
popularity_recomm.print_rows(num_rows=25)

+---------+----------+-------+------+
| user_id | movie_id | score | rank |
+---------+----------+-------+------+
|    20   |   1599   |  5.0  |  1   |
|    20   |   1201   |  5.0  |  2   |
|    20   |   1189   |  5.0  |  3   |
|    20   |   1122   |  5.0  |  4   |
|    20   |   814    |  5.0  |  5   |
|    21   |   1599   |  5.0  |  1   |
|    21   |   1201   |  5.0  |  2   |
|    21   |   1189   |  5.0  |  3   |
|    21   |   1122   |  5.0  |  4   |
|    21   |   814    |  5.0  |  5   |
|    22   |   1599   |  5.0  |  1   |
|    22   |   1201   |  5.0  |  2   |
|    22   |   1189   |  5.0  |  3   |
|    22   |   1122   |  5.0  |  4   |
|    22   |   814    |  5.0  |  5   |
|    23   |   1599   |  5.0  |  1   |
|    23   |   1201   |  5.0  |  2   |
|    23   |   1189   |  5.0  |  3   |
|    23   |   1122   |  5.0  |  4   |
|    23   |   814    |  5.0  |  5   |
|    24   |   1599   |  5.0  |  1   |
|    24   |   1201   |  5.0  |  2   |
|    24   |   1189   |  5.0  |  3   |
|    24   | 

In [12]:
#But this recommends only popular movies to the users. Not really useful.

In [16]:
#We can verify this here
ratings_train.groupby(by='movie_id')['rating'].mean().sort_values(ascending=False).head(20)

movie_id
1500    5.000000
1293    5.000000
1122    5.000000
1189    5.000000
1656    5.000000
1201    5.000000
1599    5.000000
814     5.000000
1467    5.000000
1536    5.000000
1449    4.714286
1642    4.500000
1463    4.500000
1594    4.500000
1398    4.500000
114     4.491525
408     4.480769
169     4.476636
318     4.475836
483     4.459821
Name: rating, dtype: float64

In [29]:
#Train Model
item_sim_model = graphlab.item_similarity_recommender.create(train_data, user_id='user_id', 
                                                             item_id='movie_id', target='rating', similarity_type='cosine')

#Make Recommendations:
item_sim_recomm = item_sim_model.recommend(users=range(1,6),k=5)
item_sim_recomm.print_rows(num_rows=25)

Recsys training: model = item_similarity

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.134499s

+---------+----------+----------------+------+
| user_id | movie_id |     score      | rank |
+---------+----------+----------------+------+
|    1    |   423    | 0.980611449435 |  1   |
|    1    |   202    | 0.949590799235 |  2   |
|    1    |   655    | 0.796218333126 |  3   |
|    1    |   403    | 0.765623665038 |  4   |
|    1    |   568    | 0.751179529283 |  5   |
|    2    |    50    | 1.12562584877  |  1   |
|    2    |   181    | 1.06517731685  |  2   |
|    2    |    7     | 0.999819083856 |  3   |
|    2    |   121    | 0.922513024165 |  4   |
|    2    |    9     | 0.831989913033 |  5   |
|    3    |   313    | 0.635376662016 |  1   |
|    3    |   328    | 0.603288030083 |  2   |
|    3    |   315    | 0.542258712378 |  3   |
|    3    |   331    | 0.535507185893 |  4   |
|    3    |   332    | 0.531669611281 |  5   |
|    4    |    50    | 1.13114770821  |  1   |
|    4    |   288    | 1.04871511459  |  2   |
|    4    |   181    | 0.950599938631 |  3   |
|    4    |  

In [31]:
model_performance = graphlab.compare(test_data, [popularity_model, item_sim_model])
graphlab.show_comparison(model_performance,[popularity_model, item_sim_model])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |        0.0        |        0.0        |
|   2    |        0.0        |        0.0        |
|   3    |        0.0        |        0.0        |
|   4    |        0.0        |        0.0        |
|   5    |        0.0        |        0.0        |
|   6    |        0.0        |        0.0        |
|   7    |        0.0        |        0.0        |
|   8    | 0.000132555673383 | 0.000106044538706 |
|   9    | 0.000235654530458 | 0.000212089077413 |
|   10   | 0.000212089077413 | 0.000212089077413 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+---